In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd

import coremltools as ct
from coremltools.models import MLModel

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

print("Tensorflow version:", tf.__version__)
print("Numpy version:", np.__version__)
print("Core ML tools version:", ct.__version__)

!mkdir -p 'mlmodels/'
mlmodel_url = "mlmodels/MNISTexample.mlmodel"
mlmodel_url_updatable = "mlmodels/MNISTexampleUpdatable.mlmodel"

saved_model_path = "tfmodels/coreml_mnist_numbers"
saved_lite_model_path = "tflitemodels/coreml_mnist_numbers_100.tflite"
checkpoint_path = "tfcheckpoints/coreml_checkpointmnistnumbers.ckpt"

IMPORTANT
This notebook is not running. It was used to convert a TensorFlow model into a Core ML model, which does not seem to work currently, because of incompatibilities in recent versions.

https://coremltools.readme.io/docs/comparing-ml-programs-and-neural-networks
https://github.com/apple/coremltools/issues/523
https://github.com/apple/coremltools/issues/534


In [ ]:
# create the kears model

keras_model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, input_shape=(784,), name='dense_1', activation='relu'),
            tf.keras.layers.Dense(10, name='dense_2'),
        ])


keras_model.compile(
            optimizer='adam',
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))

In [ ]:
# convert to core ml format

class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# Pass in `tf.keras.Model` to the Unified Conversion API
mlmodel = ct.convert(keras_model, convert_to="neuralnetwork")


mlmodel.save(mlmodel_url)

In [ ]:
# inspect
spec = ct.utils.load_spec(mlmodel_url)
builder = ct.models.neural_network.NeuralNetworkBuilder(spec=spec)
builder.inspect_layers(last=4)

In [ ]:
# inspect input of the model
builder.inspect_input_features()
neuralnetwork_spec = builder.spec

In [ ]:
# meta data for preview
# Set input and output description
neuralnetwork_spec.description.input[0].shortDescription = 'Input image of the handwriten digit to classify'
neuralnetwork_spec.description.output[0].shortDescription = 'Probabilities / score for each possible digit'
#neuralnetwork_spec.description.output[1].shortDescription = 'Predicted digit'

In [ ]:
# make_updatable method is used to make a layer updatable. It requires a list of layer names.
# dense_1 and dense_2 are two innerProduct layer in this example and we make them updatable.
builder.make_updatable(['sequential_2/dense_1/MatMul', 'dense_2'])

# Categorical Cross Entropy or Mean Squared Error can be chosen for the loss layer.
# Categorical Cross Entropy is used on this example. CCE requires two inputs: 'name' and 'input'.
# name must be a string and will be the name associated with the loss layer
# input must be the output of a softmax layer in the case of CCE. 
# The loss's target will be provided automatically as a part of the model's training inputs.
builder.set_categorical_cross_entropy_loss(name='lossLayer', input='digitProbabilities')

# in addition of the loss layer, an optimizer must also be defined. SGD and Adam optimizers are supported.
# SGD has been used for this example. To use SGD, one must set lr(learningRate) and batch(miniBatchSize) 
#(momentum is an optional parameter).
from coremltools.models.neural_network import SgdParams
builder.set_sgd_optimizer(SgdParams(lr=0.01, batch=32))

# Finally, the number of epochs must be set as follows.
builder.set_epochs(10)

# Set training inputs descriptions
neuralnetwork_spec.description.trainingInput[0].shortDescription = 'Example image of handwritten digit'
neuralnetwork_spec.description.trainingInput[1].shortDescription = 'Associated true label (digit) of example image'

# save the updated spec
mlmodel_updatable = MLModel(neuralnetwork_spec)
mlmodel_updatable.save(mlmodel_url_updatable)